In [ ]:
"""
Ноутбук поможет собрать и обрaботать сет из фотографий для дообучения модели распознавателя
для корректного функционировани нужно создать дополнительные папки:
1)папки с фото в которых будут лежать исходные фото. в каждой папке должны лежать фото только одного человека 
2)общая папка куда будут улетать задетектированые фото
3)общая папку куда будут улетать аугментированые фото 

Функция  get_images_and_labels забирает все из папки с иходными фото, детектирует лицо и сохранет 
в формате face-'label'-'№'.jpg например ( face-1-1.jpg) така размета заложена в файле с обучением модели

Функция augmentation аугментацию фото что повысит качество обучаемой модели

Сначала прогоняем сырые фото через  get_images_and_labels,  потом все фото через   augmentation
"""

In [49]:
import cv2
import os
import numpy as np

#импортируем детектор лиц
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")


In [50]:
def get_images_and_labels(datapath,save_path, counter,label):
    """
    Функция  get_images_and_labels забирает все из папки с иходными фото, детектирует лицо и сохранет 
    в формате face-'label'-'№'.jpg например ( face-1-1.jpg) така размета заложена в файле с обучением модели
    :param label: нужный id пользователя
    :param counter: поможет сделать перфикс для фото если нужно добавить новые фотограйии

    """
    
    
    train_paths = [os.path.join(datapath, f) for f in os.listdir(datapath)]  # получаем путь к трейн сету
    
    
    # перебираем все картинки
    for train_path in train_paths:

        image = cv2.imread(train_path)  # читаем картинку
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # переводим в ч/б

        result_frame = detector.detectMultiScale(image,
                                                 scaleFactor=1.05,
                                                 minNeighbors=6,
                                                 minSize=(100, 100),
                                                 flags=cv2.CASCADE_SCALE_IMAGE)

        for (x, y, w, h) in result_frame:
            # добавляем его к списку картинок
            cv2.imwrite(save_path + "/face-" + str(label) + '.' + str(counter) + ".jpg", image[y: y + h, x: x + w])
            counter += 1
        

    

In [3]:
#пример искользовании функции get_images_and_labels

data_path = 'eva'  # путь к датасету с фотографиями пользователей
save_path ="data_set_EVA"
counter = 200
label =2
get_images_and_labels(data_path,save_path,counter,label)

In [52]:
#пример искользовании функции get_images_and_labels
data_path = 'ilya'  # путь к датасету с фотографиями пользователей
save_path ="ilya_train_set"
counter = 300
label =1
get_images_and_labels(data_path,save_path,counter,label)

In [53]:
def rotate( img_, angle):
    """
    Функция поворота картинки на определнный угл
    :param img_: входящая картинка 
    :param angle: угол поворота
    :return: повернутая картинка
    """
    height, width = img_.shape[:2] #высота и ширина
    pivot_point =  (width // 2, height // 2) # точка поворота ( середина)
    
    mat = cv2.getRotationMatrix2D(pivot_point, angle, scale =1) #матрица поворота
    return cv2.warpAffine(img_, mat, (height, width))

In [54]:
def augmentation(datapath,save_path):
    """
    Забирает фото из папки и проводит следующие аугментации:
    поворот на 15 и 30 градусов, легкое размытие, легкую эрозию
    Возвращает в общую папку оригинал фото и аугментированые фото

    """
    kernal = np.ones((3,3), np.uint8)
    
    set_paths = [os.path.join(datapath, f) for f in os.listdir(datapath)]  # получаем путь к трейн сету
    
    
    # перебираем все картинки
    for photo_path in set_paths:
        
        base_photo_name = os.path.basename(str(photo_path)).split('jpg')[0] #вытащим базовое название фото для добавления номера аугментации

        image = cv2.imread(photo_path)  # читаем картинку
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # переводим в ч/б
        # вернем оригинал
        cv2.imwrite(save_path + base_photo_name + '0' + ".jpg", image) 
        
        # поворот на 15 и 30 гардусов
        new_image = rotate(image, 15)
        cv2.imwrite(save_path + base_photo_name + '1' + ".jpg", new_image) 
        
        new_image = rotate(image, 30)
        cv2.imwrite(save_path + base_photo_name + '2' + ".jpg", new_image)
        
        new_image = rotate(image, -15)
        cv2.imwrite(save_path + base_photo_name + '3' + ".jpg", new_image)
        
        new_image = rotate(image, -30)
        cv2.imwrite(save_path + base_photo_name + '4' + ".jpg", new_image)
        
        # размытие
        new_image = cv2.GaussianBlur(image, (9,9), 2)
        cv2.imwrite(save_path + base_photo_name + '5' + ".jpg", new_image)
        
        # эрозия
        new_image = cv2.erode(image, kernal, iterations = 1 )
        cv2.imwrite(save_path + base_photo_name + '6' + ".jpg", new_image)
        
            
        
        

In [55]:
#пример искользовании функции augmentation
data_path = 'ilya_train_set'
save_path ="data_set_advanced/"
augmentation(data_path,save_path)

In [37]:
#пример искользовании функции augmentation
data_path = 'data_set_EVA'
save_path ="test/"
augmentation(data_path,save_path)